# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
geoapify_key

'754705a009a94314b71ee0b4722d8750'

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,necochea,-38.5473,-58.7368,12.51,59,71,9.76,AR,1681348763
1,1,bredasdorp,-34.5322,20.0403,12.84,79,100,2.51,ZA,1681348280
2,2,atuona,-9.8000,-139.0333,27.03,68,100,3.58,PF,1681348284
3,3,rikitea,-23.1203,-134.9692,25.28,68,48,4.87,PF,1681348764
4,4,tuktoyaktuk,69.4541,-133.0374,-19.62,85,44,4.76,CA,1681348321


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)
    

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

#temperature
fit_criteria = city_data_df.loc[(city_data_df["Max Temp"] < 78) & (city_data_df["Max Temp"] > 70)]

#humidity
fit_criteria = city_data_df.loc[(city_data_df["Humidity"] < 41) & (city_data_df["Max Temp"] > 35)]

#wind speed
fit_criteria = city_data_df.loc[city_data_df["Wind Speed"] < 8]

# Drop any rows with null values
fit_criteria_clear = fit_criteria.dropna()

# Display sample data
fit_criteria_clear

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,bredasdorp,-34.5322,20.0403,12.84,79,100,2.51,ZA,1681348280
2,2,atuona,-9.8000,-139.0333,27.03,68,100,3.58,PF,1681348284
3,3,rikitea,-23.1203,-134.9692,25.28,68,48,4.87,PF,1681348764
4,4,tuktoyaktuk,69.4541,-133.0374,-19.62,85,44,4.76,CA,1681348321
6,6,mataura,-46.1927,168.8643,11.38,77,95,3.74,NZ,1681348764
...,...,...,...,...,...,...,...,...,...,...
585,585,champerico,14.3000,-91.9167,29.35,63,100,2.75,GT,1681348330
586,586,isoka,-10.1606,32.6335,17.38,99,100,0.96,ZM,1681348392
588,588,puerto baquerizo moreno,-0.9000,-89.6000,26.55,83,40,1.03,EC,1681348347
590,590,honiara,-9.4333,159.9500,29.48,70,41,2.05,SB,1681348384


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,necochea,-38.5473,-58.7368,12.51,59,71,9.76,AR,1681348763,
1,1,bredasdorp,-34.5322,20.0403,12.84,79,100,2.51,ZA,1681348280,
2,2,atuona,-9.8000,-139.0333,27.03,68,100,3.58,PF,1681348284,
3,3,rikitea,-23.1203,-134.9692,25.28,68,48,4.87,PF,1681348764,
4,4,tuktoyaktuk,69.4541,-133.0374,-19.62,85,44,4.76,CA,1681348321,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 5000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
necochea - nearest hotel: Cabañas Río Quequén
bredasdorp - nearest hotel: Victoria Hotel
atuona - nearest hotel: Pearl Resort
rikitea - nearest hotel: Chez Bianca & Benoit
tuktoyaktuk - nearest hotel: Hunter's B&B
umm kaddadah - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
port blair - nearest hotel: Andaman Ocean Breeze
roald - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
avera - nearest hotel: No hotel found
hualmay - nearest hotel: Hotel Mi Choacan
georgetown - nearest hotel: Page 63 hostel
albany - nearest hotel: No hotel found
butaritari - nearest hotel: No hotel found
dejen - nearest hotel: Maflab
souillac - nearest hotel: Hotel SkylineInn
tapes - nearest hotel: No hotel found
kapaa - nearest hotel: Pono Kai Resort
hithadhoo - nearest hotel: Pebbles Inn
bethel - nearest hotel: Hampton Inn Danbury
sinnamary - nearest hotel: Hôtel du fleuve
krasnyy chikoy - nearest hotel: Гостиница ЗАО "Слюдянка"
sola - nea

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,necochea,-38.5473,-58.7368,12.51,59,71,9.76,AR,1681348763,Cabañas Río Quequén
1,1,bredasdorp,-34.5322,20.0403,12.84,79,100,2.51,ZA,1681348280,Victoria Hotel
2,2,atuona,-9.8000,-139.0333,27.03,68,100,3.58,PF,1681348284,Pearl Resort
3,3,rikitea,-23.1203,-134.9692,25.28,68,48,4.87,PF,1681348764,Chez Bianca & Benoit
4,4,tuktoyaktuk,69.4541,-133.0374,-19.62,85,44,4.76,CA,1681348321,Hunter's B&B
...,...,...,...,...,...,...,...,...,...,...,...
587,587,campbeltown,55.4258,-5.6076,6.03,87,99,13.32,GB,1681348833,The Royal Hotel
588,588,puerto baquerizo moreno,-0.9000,-89.6000,26.55,83,40,1.03,EC,1681348347,Dory's House
589,589,warrington,53.3925,-2.5802,7.93,86,75,12.86,GB,1681348380,Travelodge
590,590,honiara,-9.4333,159.9500,29.48,70,41,2.05,SB,1681348384,King Solomon Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
%%capture --no-display

# Configure the map plot
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500,
    scale = 0.7,
    color = "City",
    hover_cols = ['Country', 'Hotel Name']
)
    

# # Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)